### 0. Data structure
#### 0.1 heap & 优先队列
* 首先是队列
* 其次其访问次序不是FIFO, 而是根据定义的优先级来排序

##### 0.1.1 应用模式
```
server       <----           client requests
  |                                 |
scheduler  <--- priority queue   <-- importance priority   
```
* 优先队列需要动态的维护优先级次序，
* 会有很多插入，删除，排序
* 接口：get/insert(put)/delete
* stack/queue 都是优先队列的特殊情况

##### 0.1.2 基本实现
|结构|get_max| del_max |insert|
|----|---|---|---|
|vector|O(n)|O(n)|O(1)|
|sorted vector|O(1)|O(1)|O(n)
|list|O(n)|O(n)|O(1)
|BBST|log(n)|log(n)|log(n)
|完全二叉树/堆|O(1)|log(n)|log(n)

* blanced binary tree 的效率其实是被浪费了，因为永远只get or del **MAX**, 而bbst始终维护全序关系，只用了2/3的BBST

##### 0.1.3 完全二叉树实现 <=> 完全二叉堆
* 逻辑上，等同于完全二叉树
* 物理上用vector实现

![Complete binary tree - vector impletementation](img/complete_bt.png)

###### 0.1.4 堆序性
* H[I] <= H[Parent(i)]
* 堆序性保证，根节点是最大的节点。
* get_max 就肯定是vector[0]

##### 0.1.5 完全二叉堆 插入与堆序性
* popup, 如大于parent，则递归的和parent互换
* log(n)

##### 0.1.6 完全二叉堆 删除与堆序性
* top-down 下滤，把末尾元素popup到定，然后把top元素向下换

In [74]:
import heapq
nums = [3,10,1000,-99,4,100]

* heapify 拿一个iterable 变成一个逻辑上的二叉堆

In [75]:
heapq.heapify(nums)
print(nums)

[-99, 3, 100, 10, 4, 1000]


In [36]:
heapq.heappop(nums)

-99

#### 0.2 双向队列 - deque
* deque是双端队列，可以在两端扩展和收缩
* 允许元素随机读取


* deque 像是一个linked-list 和 vector的混合产品，双端插入删除效率高，且支持随机读取

In [84]:
dq = deque()
dq.extend([2, 3, 4])

dq.append(5)
dq.appendleft(1)

* deque 随机存取

In [85]:
dq[2]

3

In [86]:
print(dq.pop())
print(dq.popleft())

5
1


#### 0.3 ordereddict - BST

* 利用ordereddict实现LRU

In [89]:
from collections import OrderedDict
class LRU(OrderedDict):
    'Limit size, evicting the least recently looked-up key when full'

    def __init__(self, maxsize=128, *args, **kwds):
        self.maxsize = maxsize
        super().__init__(*args, **kwds)

    def __getitem__(self, key):
        value = super().__getitem__(key)
        self.move_to_end(key)
        return value

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        if len(self) > self.maxsize:
            oldest = next(iter(self))
            del self[oldest]

#####  0.4 bisect
模块能够提供保持list元素序列的支持。它使用了二分法完成大部分的工作。它在向一个list插入元素的同时维持list是有序的。在某些情况下，这比重复的对一个list进行排序更为高效，并且对于一个较大的list来说，对每步操作维持其有序也比对其排序要高效。

##### 0.5 counter

##### 0.6 Weakref
weakref模块能够帮助我们创建Python引用，却不会阻止对象的销毁操作

### I. 元编程
#### 1.1 装饰器
##### 1.1.1 装饰函数
下面两个例子等价

In [4]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    def method(cls):
        pass
    method = classmethod(method)

* 用wraps decorator保留函数的元信息，名字，文档，签名etc

In [8]:
from functools import wraps
import time

def timethis(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print("Running time ", time.time() - start)
        return result
    
    return wrapper

In [9]:
@timethis
def run(app_name):
    '''start a app'''
    time.sleep(1)
    return app_name

In [10]:
run("app")
print(run.__name__, run.__doc__)

Running time  1.0004971027374268
run start a app


#### 1.1.2 装饰一个类

In [11]:
import types

class Profiled:
    def __init__(self, func):
        self.wrapped = wraps(func)(self)
        self.ncalls = 0
    
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.wrapped(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self 
        else:
            return types.MethodType(self, instance)

In [12]:
@Profiled
def add(x, y):
    return x + y

In [13]:
# add(1, 2)

#### 1.2 Descriptor
描述符是实现了特定协议的类，包括：\__get\__, \__set\__, \__delete\__

在实现 保护属性不受修改 、 属性类型检查 的基本功能，同时有大大提高代码的复用率。

##### 1.2.1作用
* 弥补property的不足, 如果用property实现下面的功能，那么我们需要2组setter去分别检查weight，price，不能复用
* 设置属性不能被删除？那定义_delete_方法，并raise 异常。 
* 还可以设置只读属性，\__set\__, raise exception


In [14]:
class Quantity(object):

    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __get__(self, instance, owner):
        # return instance.__dict__[self.storage_name]
        return getattr(self, self.storage_name)

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError("value must possive")

        # instance.__dict__[self.storage_name] = value
        setattr(self, self.storage_name, value)


class LineItem(object):
    weight = Quantity("weight")
    price = Quantity("price")

    def __init__(self, weight, price):
        self.weight = weight
        self.price = price

In [15]:
l1 = LineItem(1, 2)
print(l1.weight, l1.price)

# l2 = LineItem(-1, 0)   ValueError: value must possive

1 2


#### 1.3 元类
* metaclass这个词，模糊地描述了一种高于类，而又超乎类的概念。
* 我们把python的class语句转义为元类，令其在定义类型时都会提供独特的行为

##### 1.3.1 用元类来验证子类
* 通过元类，可以再生成子类对象之前，定义类时，先验证子类的定义是否合乎规范
* py2/3语法不同
* python把子类的整个class语句体处理完，就会调用元类的\__new\__方法

In [36]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print("In Meta: ", meta, name, bases, class_dict)
        
        if "foo" not in class_dict:
            print("Invalid class for Meta")
            
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(object, metaclass=Meta):
    print("BEFORE")
    def foo(self):
        pass
    print("AFTER")

BEFORE
AFTER
In Meta:  <class '__main__.Meta'> MyClass (<class 'object'>,) {'__module__': '__main__', 'foo': <function MyClass.foo at 0x00000199E77D1840>, '__qualname__': 'MyClass'}


In [37]:
class MyClass2(object, metaclass=Meta):
    pass

In Meta:  <class '__main__.Meta'> MyClass2 (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'MyClass2'}
Invalid class for Meta


##### 1.3.2 用元类来注册子类
* 可以在程序中自动注册类型，在需要方向查找(reverse lookup)的场合，非常有用
* 可以在简单的标识符与对应的类之间建立映射关系


* 在下例中需要一个registry去维护需要被deserialize的类名，
* 就可以不用 BetterPoint2D.deserialize(data) 这种形式，而是更generic的形式来调用deserialize方法
* 需要把register_class 放在一个地方，不会被忘记调用

In [41]:
import json

registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class
    
    
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        # 真的在内存里实例化这个实例
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)         # 注册类！
        return cls

    
def deserialize(data):
    params = json.loads(data)
    target_class = registry[params["class"]]
    return target_class(*params["args"])
    
    
class Serializable(object):
    
    def __init__(self, *args):
        self.args = args
    
    def serialize(self):
        return json.dumps({
                'class': self.__class__.__name__,
                'args':self.args})
    
       
class Point2D(Serializable, metaclass=Meta):
    
    def __init__(self, x, y):
        super(Point2D, self).__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)

* Point2D 继承Serizlizable，但是元类时Meta

In [42]:
point = Point2D(2, 4)
data = point.serialize()
print(data)

{"class": "Point2D", "args": [2, 4]}


In [40]:
print("After deserialize", deserialize(data))

After deserialize Point(2, 4)


##### 1.3.2 用元类来注解类的属性

### II. 网络编程

```python
import multiprocessing
from multiprocessing.reduction import recv_handle, send_handle
import socket

def worker(in_p, out_p):
    out_p.close()
    
    while True:
        fd = recv_handle(in_p)
        print("CHILD: GOT File Discriptor: ", fd)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM, fileno=fd) as s:
            with True:
                msg = s.recv(1024)
                if not msg:
                    break
                print("CHILD RECV {!r}".format(msg))
                s.send(msg)
        
def server(address, in_p, out_p, worker_pid):
    in_p.close()
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
    s.bind(address)
    s.listen(1)
    
    while True:
        client, addr = s.accept()
        print("SERVER: Got connection from ", addr)
        send_handle(out_p, client.fileno(), worker_pid)
        client.close()

if __name__ == "__main__":
    c1, c2 = multiprocessing.Pipe()

    worker_p = multiprocessing.Process(target=worker, args=(c1, c2))
    worker_p.start()

    server_p = multiprocessing.Process(target=server, args=(('', 15000), c1, c2, worker_p.pid))
    server_p.start()

    c1.close()
    c2.close()
```

### III. 并发编程

#### 协程 - coroutine 
绿色线程，轻量级的线程。

* 示例1，用coroutine代替线程

In [1]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        yield
        n -= 1
    print('Blastoff!')
    
    
def countup(n):
    x = 0
    while x < n:
        print('Conuting up', x)
        yield
        x += 1

In [2]:
from collections import deque

class TaskScheduler:
    
    def __init__(self):
        self._task_queue = deque()
        
    def new_task(self, task):
        '''Admit a newly started task to the scheduler'''
        self._task_queue.append(task)
    
    def run(self):
        while self._task_queue:
            task = self._task_queue.popleft()
            
            try:
                next(task)
                self._task_queue.append(task) # 轮询的运行queue里的协程
            except StopIteration:
                pass

In [3]:
scheduler = TaskScheduler()
scheduler.new_task(countdown(5))
scheduler.new_task(countup(6))
scheduler.run()

T-minus 5
Conuting up 0
T-minus 4
Conuting up 1
T-minus 3
Conuting up 2
T-minus 2
Conuting up 3
T-minus 1
Conuting up 4
Blastoff!
Conuting up 5


* 示例2

In [6]:
class ActorScheduler:
    
    def __init__(self):
        self._actors = {}
        self._msg_queue = deque()
        
    def new_actor(self, name, actor):
        self._msg_queue.append((actor, None))
        self._actors[name] = actor
       
    def send(self, name, msg):
        actor = self._actors.get(name)
        if actor:
            self._msg_queue.append((actor, msg))
            
    def run(self):
        while self._msg_queue:
            actor, msg = self._msg_queue.popleft()
            
            try:
                actor.send(msg)
                print("Sending ", msg, " to ", actor.__name__)
            except StopIteration:
                pass

In [7]:
def printer():
    while True:
        msg = yield
        print('Got: ', msg)

        
def counter(sched):
    
    while True:
        n = yield
        if n == 0:
            break
        
        sched.send('printer', n)
        
        sched.send('counter', n-1)

In [8]:
sched = ActorScheduler()
sched.new_actor('printer', printer())
sched.new_actor('counter', counter(sched))

sched.send('counter', 3)
sched.run()

Sending  None  to  printer
Sending  None  to  counter
Sending  3  to  counter
Got:  3
Sending  3  to  printer
Sending  2  to  counter
Got:  2
Sending  2  to  printer
Sending  1  to  counter
Got:  1
Sending  1  to  printer


* 3

In [1]:
from select import select
from collections import deque

class YieldEvent:
    def handle_yield(self, sched, task):
        pass
    
    def handle_resume(self, sched, task):
        pass

    
class Scheduler:
    
    def __init__(self):
        self._numtasks = 0
        self._ready = deque()
        self._read_waiting = {}
        self._write_waiting = {}
    
    def _iopoll(self):
        rset, wset, eset = select(self._read_waiting, 
                                 self._write_waiting, [])
        
        for r in rset:
            evt, task = self._read_waiting.pop(r)
            evt.handle_resume(self, task)
        
        for w in wset:
            evt, task = self._write_waiting.pop(w)
            evt.handle_resume(self, task)
    
    def new(self, task):
        self._ready.append((task, None))
        self._numtasks += 1
    
    def add_ready(self, task, msg=None):
        self._ready.append((task, msg))
        
    def _read_wait(self, fileno, evt, task):
        self._read_waiting[fileno] = (evt, task)
    
    def _write_wait(self, fileno, evt, task):
        self._write_waiting[fileno] = (evt, task)
    
    def run(self):    
        while self._numtasks:
        
            if not self._ready:
                self._iopoll()
            task, msg = self._ready.popleft()
            
            try:
                r = task.send(msg)
                if isinstance(r, YieldEvent):
                    r.handle_yield(self, task)
                else:
                    raise RuntimeError('Unrecognized yield event')
            except StopIteration:
                self._numtasks -= 1

In [2]:
class ReadSocket(YieldEvent):
    
    def __init__(self, sock, nbytes):
        self.sock = sock
        self.nbytes = nbytes
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        data = self.sock.recv(self.nbytes)   # 执行socket listen
        print("recieved data ", data)
        sched.add_ready(task, data)
    
class WriteSocket(YieldEvent):
    
    def __init__(self, sock, data):
        self.sock = sock
        self.data = data
    
    def handle_yield(self, sched, task):
        sched._write_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        nsent = self.sock.send(self.data)   # 执行 socket send
        sched.add_ready(task, nsent)
    
class AcceptSocket(YieldEvent):
    
    def __init__(self, sock):
        self.sock = sock
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        r = self.sock.accept()   # 执行 socket accept
        sched.add_ready(task, r)
    
class Socket(object):
    
    def __init__(self, sock):
        self._sock = sock
    
    def recv(self, maxbytes):
        return ReadSocket(self._sock, maxbytes)
    
    def send(self, data):
        return WriteSocket(self._sock, data)
    
    def accept(self):
        return AcceptSocket(self._sock)
    
    def __getattr__(self, name):
        return getattr(self._sock, name)

In [3]:
from socket import socket, AF_INET, SOCK_STREAM
import time


def readline(sock):
    chars = []
    while True:
        c = yield sock.recv(1)
        if not c:
            break
        
        chars.append(c)
        if c == b'\n':
            break
    return b''.join(chars)


class EchoServer:
    
    def __init__(self, addr, sched):
        self.sched = sched
        sched.new(self.server_loop(addr))
    
    def server_loop(self, addr):
        s = Socket(socket(AF_INET, SOCK_STREAM))
        
        s.bind(addr)
        s.listen(5)
        
        while True:
            c, a = yield s.accept()
            print('Got connnection from ', a)
            self.sched.new(self.client_handler(Socket(c)))
        
    def client_handler(self, client):
        while True:
            line = yield from readline(client)
            if not line:
                break
            
            line = b'Got: ' + line
            while line:
                nsent = yield client.send(line)
                line = line[nsent:]
            
            client.close()
            print('Client closed')

In [11]:
sched = Scheduler()
EchoServer(('', 1600), sched)
sched.run()

NameError: name 'Scheduler' is not defined

##### 多进程
* multiprocessing
* current.futures

In [1]:
from multiprocessing import Pipe
a, b = Pipe()
a.send([1, 'hello', None])

[1, 'hello', None]